In [3]:
import ollama 
import os
import pandas as pd
import numpy as np
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_core.documents import Document
from langchain_postgres.vectorstores import PGVector
from sqlalchemy import  URL ,create_engine


In [4]:
# We can use also opensource embeddings models run
# from langchain.embeddings.ollama import OllamaEmbeddings
# embeddings = OllamaEmbeddings(
#     model="llama3.1",
# )
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [5]:
loader = PyPDFLoader("constitucion_colombia.pdf")
documents = loader.load()

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=2500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [8]:
connection_string = URL.create( drivername="postgresql", database="postgres", username="postgres", password="julian1234", host="localhost", port="5432", )
engine = create_engine(connection_string)
COLLECTION_NAME = "Constitucion"

In [9]:
# Add embeddings in the database
vector_store = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection=engine,
    pre_delete_collection=False,
)

In [10]:
pgvector_docsearch = PGVector(
    collection_name=COLLECTION_NAME,
    connection=engine,
    embeddings=embeddings,
    use_jsonb=True,
)

In [14]:
retriever = pgvector_docsearch.as_retriever()
retriever.invoke("Que derechos tienen los colombianos?")

[Document(metadata={'page': 36, 'source': 'constitucion_colombia.pdf'}, page_content='serán los colombianos nacionalizados en país extranjero, contra el país de su \nnueva nacionalidad.  \nCAPÍTULO 2 \nDE LA CIUDADANÍ A \nArtículo 98. La ciudadanía se pierde de hecho cuando se ha renunciado a la \nnacionalidad, y su ejercicio se puede suspender en virtud de decisión judicial en \nlos casos que determine la ley.  \nQuienes hayan sido suspendidos en el ejercicio de la ciudadanía, podrán solicitar \nsu rehabilitación.  \nParágrafo . Mientras la ley no decida otra edad, la ciudadanía se ejercerá a partir \nde los dieciocho años.  \nArtículo 99. La calidad de ciudadano en ejercicio es condición previa e \nindispensable para ejercer el derecho de sufragio, para ser elegido y para \ndesempeñar cargos públicos que lleven anexa autoridad o jurisdicción.  \nCAPÍTULO 3  \nDE LOS EXTRANJEROS  \nArtículo 100. Los ext ranjeros disfrutarán en Colombia de los mismos derechos \nciviles que se conceden 

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())
setup.invoke("Quienes son libres en colombia?")

{'context': [Document(metadata={'page': 16, 'source': 'constitucion_colombia.pdf'}, page_content='Artículo  16. Todas  las personas tienen derecho al libre desarrollo de su \npersonalidad sin más limitaciones que las que imponen los derechos de los demás \ny el orden jurídico.  \nArtículo  17. Se prohíben la esclavitud, la servidumbre y la trata de seres humanos \nen todas sus formas.  \nArtícul o 18. Se garantiza la libertad de conciencia. Nadie será molestado por \nrazón de sus convicciones o creencias ni compelido a revelarlas ni obligado a \nactuar contra su conciencia.  \nArtículo  19. Se garantiza la libertad de cultos. Toda persona t iene derecho a \nprofesar libremente su religión y a difundirla en forma individual o colectiva.  \nTodas las confesiones religiosas e iglesias son igualmente libres ante la ley.  \nArtículo  20. Se garantiza a toda persona la libertad de expresar y difundir su \npensamiento y opiniones, la de informar y recibir información veraz e imparcial, y \nla

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models import ChatOpenAI

parser = StrOutputParser()
llm = ChatOpenAI(model='gpt-4o-mini-2024-07-18')


from langchain.prompts import ChatPromptTemplate

template = """
Responde las preguntas que te hagan basandote en el contexto, 
eres un asistente que responde preguntas sobre la consitutcion politica de colombia
Context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = setup | prompt | llm | parser

In [17]:
chain.invoke("Que personas son libres en colombia?")

'En Colombia, todas las personas son libres en el sentido de que tienen derecho al libre desarrollo de su personalidad, según lo establecido en el Artículo 16 de la Constitución Política. Esto significa que cada individuo tiene la libertad de actuar conforme a su propia personalidad, siempre y cuando no infrinja los derechos de los demás y se ajuste al orden jurídico. Además, se garantiza la libertad de conciencia (Artículo 18), la libertad de cultos (Artículo 19), y la libertad de expresar y difundir pensamientos y opiniones (Artículo 20), lo que refuerza la idea de que todas las personas son libres en el país.'